In [1]:
import pandas as pd
import seaborn as sns
import json

In [2]:
cards_df = pd.read_csv('cards_data.csv')
trans_df = pd.read_csv('transactions_data.csv')
users_df = pd.read_csv('users_data.csv')

In [3]:
with open('mcc_codes.json', 'r') as f:
    mcc_data = json.load(f)

with open('train_fraud_labels.json', 'r') as f:
    fraud_labels = json.load(f)

label_data = fraud_labels["target"]

In [4]:
print('User Columns:', users_df.columns)
print('-'*100)
print('Cards Columns:', cards_df.columns)
print('-'*100)
print('Transactions Columns:', trans_df.columns)
print('-'*100)

User Columns: Index(['id', 'current_age', 'retirement_age', 'birth_year', 'birth_month',
       'gender', 'address', 'latitude', 'longitude', 'per_capita_income',
       'yearly_income', 'total_debt', 'credit_score', 'num_credit_cards'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Cards Columns: Index(['id', 'client_id', 'card_brand', 'card_type', 'card_number', 'expires',
       'cvv', 'has_chip', 'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Transactions Columns: Index(['id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors'],
      dtype='object')
-----------------------------------------------------------------------------------------

In [5]:
print('User Data Types:', users_df.dtypes)
print('-'*100)
print('Cards Data Types:', cards_df.dtypes)
print('-'*100)
print('Transactions Data Types:', trans_df.dtypes)
print('-'*100)

User Data Types: id                     int64
current_age            int64
retirement_age         int64
birth_year             int64
birth_month            int64
gender                object
address               object
latitude             float64
longitude            float64
per_capita_income     object
yearly_income         object
total_debt            object
credit_score           int64
num_credit_cards       int64
dtype: object
----------------------------------------------------------------------------------------------------
Cards Data Types: id                        int64
client_id                 int64
card_brand               object
card_type                object
card_number               int64
expires                  object
cvv                       int64
has_chip                 object
num_cards_issued          int64
credit_limit             object
acct_open_date           object
year_pin_last_changed     int64
card_on_dark_web         object
dtype: object
--------------

In [6]:
print('User Null Values:', users_df.isna().sum())
print('-'*100)
print('Cards Null Values:', cards_df.isna().sum())
print('-'*100)
print('Transactions Null Values:', trans_df.isna().sum())
print('-'*100)

User Null Values: id                   0
current_age          0
retirement_age       0
birth_year           0
birth_month          0
gender               0
address              0
latitude             0
longitude            0
per_capita_income    0
yearly_income        0
total_debt           0
credit_score         0
num_credit_cards     0
dtype: int64
----------------------------------------------------------------------------------------------------
Cards Null Values: id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
dtype: int64
----------------------------------------------------------------------------------------------------
Transactions Null Values: id                       0
date     

## Data Cleaning on Transaction Data

In [7]:
trans_df['errors'].unique()

array([nan, 'Technical Glitch', 'Bad Expiration', 'Bad Card Number',
       'Insufficient Balance', 'Bad PIN', 'Bad CVV', 'Bad Zipcode',
       'Insufficient Balance,Technical Glitch',
       'Bad PIN,Insufficient Balance', 'Bad PIN,Technical Glitch',
       'Bad Expiration,Technical Glitch',
       'Bad Card Number,Bad Expiration',
       'Bad Card Number,Insufficient Balance',
       'Bad Expiration,Insufficient Balance', 'Bad Card Number,Bad CVV',
       'Bad CVV,Technical Glitch', 'Bad Expiration,Bad CVV',
       'Bad CVV,Insufficient Balance', 'Bad Card Number,Technical Glitch',
       'Bad Zipcode,Insufficient Balance',
       'Bad Card Number,Bad Expiration,Insufficient Balance',
       'Bad Zipcode,Technical Glitch'], dtype=object)

In [9]:
trans_df['errors'].value_counts()

errors
Insufficient Balance                                   130902
Bad PIN                                                 32119
Technical Glitch                                        26271
Bad Card Number                                          7767
Bad Expiration                                           6161
Bad CVV                                                  6106
Bad Zipcode                                              1126
Bad PIN,Insufficient Balance                              293
Insufficient Balance,Technical Glitch                     243
Bad Card Number,Insufficient Balance                       71
Bad PIN,Technical Glitch                                   70
Bad CVV,Insufficient Balance                               57
Bad Expiration,Insufficient Balance                        47
Bad Card Number,Bad CVV                                    38
Bad Card Number,Bad Expiration                             33
Bad Expiration,Bad CVV                                     32
B

In [10]:
print('Null percentage in errors column:', str(round((trans_df['errors'].isna().sum()/trans_df.shape[0])*100,2))+'%')

Null percentage in errors column: 98.41%


- We have 98.4% of null values in the 'errors' column
- We can utilize existing error messages and create new columns based on assumptions

In [ ]:
# Let's create Binary column whether it has error message or not --- "has_error"

trans_df['has_error'] = trans_df['errors'].notnull().astype(int)


- Let's assume,  there is a less chance of user making a mistake in typing these values: Bad PIN, Bad Card Number, Bad Expiration, Bad CVV, Bad Zipcode
- We can consider these values as 'suspicious_error'

In [17]:
suspicious = ['Bad PIN', 'Bad Card Number', 'Bad CVV', 'Bad Expiration', 'Bad Zipcode']
trans_df['suspicious_error'] = trans_df['errors'].apply(lambda x: 1 if isinstance(x, str) and any(err in x for err in suspicious) else 0)


In [19]:
trans_df.head(2)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,mcc_name,fraud_label,has_error,suspicious_error
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,Miscellaneous Food Stores,No,0,0
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN,Department Stores,No,0,0


In [20]:
trans_df['mcc'] = trans_df['mcc'].astype(str)
trans_df['mcc_name'] = trans_df['mcc'].map(mcc_data)

trans_df['id'] = trans_df['id'].astype(str)
trans_df['fraud_label'] = trans_df['id'].map(label_data)
trans_df.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,mcc_name,fraud_label,has_error,suspicious_error
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,Miscellaneous Food Stores,No,0,0
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN,Department Stores,No,0,0
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN,Money Transfer,No,0,0
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN,Money Transfer,NaN,0,0
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN,Drinking Places (Alcoholic Beverages),No,0,0


In [24]:
trans_df = trans_df[trans_df['fraud_label'].notnull()]

In [27]:
trans_df.to_csv('preprocessed_transaction_data.csv')

In [3]:
trans_df = pd.read_csv('preprocessed_transaction_data.csv')

In [4]:
trans_df.head()

,Unnamed: 0,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,mcc_name,fraud_label,has_error,suspicious_error
0,0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,Miscellaneous Food Stores,No,0,0
1,1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN,Department Stores,No,0,0
2,2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN,Money Transfer,No,0,0
3,4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN,Drinking Places (Alcoholic Beverages),No,0,0
4,5,7475333,2010-01-01 00:07:00,1807,165,$4.81,Swipe Transaction,20519,Bronx,NY,10464.0,5942,NaN,Book Stores,No,0,0


In [5]:
trans_df.shape

(8914963, 17)

In [6]:
# Removing 'errors', 'zip' columns
trans_df.drop(['errors', 'zip'], axis=1, inplace=True)

In [7]:
# Using the exisiting merchant city and merchant_state, replacing the null values in merchant state column
city_state_map = trans_df.dropna(subset=['merchant_state']).groupby('merchant_city')['merchant_state'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').to_dict()

trans_df['merchant_state'] = trans_df.apply(
    lambda row: city_state_map.get(row['merchant_state'], 'Unknown') if pd.isnull(row['merchant_state']) else row['merchant_state'],
    axis=1
)

trans_df['merchant_state'].value_counts()

merchant_state
Unknown       1047865
CA             956356
TX             677139
NY             574521
FL             469600
               ...   
Gabon               2
Suriname            2
Tonga               1
Ethiopia            1
Azerbaijan          1
Name: count, Length: 200, dtype: int64

- We can observe there are still unknown merchant state values, we will have "unknown" as one separate category. 
- These "unknown" values can be due to international transactions. 

In [8]:
trans_df.isna().sum()

Unnamed: 0          0
id                  0
date                0
client_id           0
card_id             0
amount              0
use_chip            0
merchant_id         0
merchant_city       0
merchant_state      0
mcc                 0
mcc_name            0
fraud_label         0
has_error           0
suspicious_error    0
dtype: int64

# Combining Data

In [ ]:
# Combining users, card, and transaction data
merged_df = pd.merge(trans_df, users_df, left_on='client_id', right_on='id', how='left')
merged_df.drop(columns=['id_x', 'Unnamed: 0', 'id_y'], inplace=True)
final_df = pd.merge(merged_df, cards_df, left_on='card_id', right_on='id', how='left')

In [17]:
final_df.columns

Index(['date', 'client_id_x', 'card_id', 'amount', 'use_chip', 'merchant_id',
       'merchant_city', 'merchant_state', 'mcc', 'mcc_name', 'fraud_label',
       'has_error', 'suspicious_error', 'current_age', 'retirement_age',
       'birth_year', 'birth_month', 'gender', 'address', 'latitude',
       'longitude', 'per_capita_income', 'yearly_income', 'total_debt',
       'credit_score', 'num_credit_cards', 'id', 'client_id_y', 'card_brand',
       'card_type', 'card_number', 'expires', 'cvv', 'has_chip',
       'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'card_on_dark_web'],
      dtype='object')

In [ ]:
# Replacing "$" and ',' with empty space
def clean_currency(val):
    if isinstance(val, str):
        return val.replace("$", "").replace(",", "")
    return val


numeric_cols = ['amount', 'per_capita_income', 'yearly_income', 'total_debt', 'credit_limit']
for cols in numeric_cols:
    final_df[cols] = final_df[cols].apply(clean_currency).astype(float)

In [20]:
final_df.head(2)

,date,client_id_x,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,mcc,mcc_name,...,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,5499,Miscellaneous Food Stores,...,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,55.0,05/2008,2008,No
1,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,5311,Department Stores,...,Credit,5175842699412235,12/2024,438,YES,1,9100.0,09/2005,2015,No
